In [1]:
# Importing relevant packages and establishing the Exadata connection

import sqlalchemy
import pandas as pd
import numpy as np
import getpass

password=getpass.getpass("Enter your password:")
engine = sqlalchemy.create_engine("oracle+cx_oracle://anantk[an_ct_ws04]:%s@exa_gbtukprdd_an" %(password))


Enter your password:········


In [12]:
import datetime as dt

# Getting the model data set from Exadata 

# prods = pd.read_sql("SELECT  distinct prod_buyer_l24_desc \
#                      from prod_dim_c " , engine )

# CREATE TABLE BB_PROD AS 
# SELECT A.*,B.PROD_CODE,PROD_GROUP_DESC,CONVERTED_PROD_GROUP_ID AS TPNB FROM BRANDBANK_INGREDIENTS_V3 
# A INNER JOIN PROD_DIM_C B ON A.EAN = CONVERTED_PROD_ID ;

# CREATE TABLE BB_PROD_2 AS 
# SELECT DISTINCT TPN, PROD_GROUP_DESC,CATEGORY,
#          rtrim(xmlagg(XMLELEMENT(E,INGREDIENTS,' ').EXTRACT('//text()')).GetClobVal(),' ') very_long_text
# --         LISTAGG(INGREDIENTS, ' ') WITHIN GROUP (ORDER BY INGREDIENTS) AS INGRE_DESC
#     FROM (SELECT DISTINCT TPN, PROD_GROUP_DESC,CATEGORY, INGREDIENTS FROM BB_PROD_1) 
# GROUP BY TPN, PROD_GROUP_DESC,CATEGORY;

# prods = pd.read_sql("CREATE TABLE AK_PROD_TM_N AS SELECT * FROM \
#                     (SELECT  distinct converted_prod_group_id as tpn, \
#                     row_number() over(partition by converted_prod_group_id order by converted_prod_group_id) as rown,\ 
#                     prod_group_code as prod,brand_name,prod_group_desc,prod_buyer_l23_desc as category from prod_dim_c a \
#                     inner join (select distinct prod_id from transaction_item_fct \
#                                   where date_id between '01-JUN-2016' and '25-JUL-16') b \ 
#                     on a.prod_id = b.prod_id where \
#                     prod_buyer_l24_desc in ('PACKAGE D.','FRES H.') ) where rown = 1" , engine )

# sprods = pd.read_csv('/nfs/science/shared/ipythonNotebooks/anantk/prod_match_up2.csv',header = 0 )
# print(sprods.head())


prods = pd.read_sql("SELECT  a.*,dense_rank()over(order by category) as cat_code FROM AK_PROD_TM_N2 a\
                     order by match_flag desc" , engine )

# prods_ingr = pd.read_sql("SELECT  * FROM BB_PROD_2 " , engine )

# print(prods_ingr.head())
# print(prods.describe())

In [6]:
import re as re
import nltk
from nltk.stem.porter import *
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

toker = TreebankWordTokenizer()

# lemmer = wordnet.WordNetLemmatizer()
stemmer = SnowballStemmer("english")
# prods['brand'] = prods.prod_group_desc.str.split(' ', 1,expand = True)[0]
prods_ingr['short_desc'] = prods_ingr.very_long_text.str.lower()

def text_preprocessor(x):

    tmp = str(x)
    tmp = tmp.lower().replace('blu-ray', 'bluray').replace('wi-fi', 'wifi')
    x_cleaned = tmp.replace('/', ' ').replace('-', ' ').replace('"', '')
    
    stops = stopwords.words("english")    
#     # 
#     # 5. Remove stop words
    # 6. Join the words back into one string separated by space, and return the result.
#     return( " ".join( meaningful_words ))
    
    tokens = toker.tokenize(x_cleaned)
    meaningful_words = [w for w in tokens if not w in [stops, 'to', 'serve', 'decorate', 'for', 'cut', 'snipped', 'tbsp','extra','litre','and','h',
                                                       'multi','multipack','eday','original','regular','single','twin','f','l','a','light','s','p','n','t','m',
                                                        'virgin','pk','c','x','w','pouch','finest', 'everyday', 'pack','value', 'fresh', 'small', 'large', 'medium']] 
    
#     return " ".join([stemmer.stem(z) for z in tokens])
    return( " ".join( meaningful_words ))


prods_ingr['desc']  = prods_ingr['short_desc'].apply(text_preprocessor)
# prods
# prods['brand'] = prods.prod_group_desc.str.split(' ', 1,expand = True)[0]
# prods['short_desc'] = prods.prod_group_desc.str.lower()
# prods_token=nltk.word_tokenize(prods.short_desc).str()

# prods['brand'] = prods.prod_group_desc.str.split(' ', 1,expand = True)[0]
# prods['short_desc'] = prods.prod_group_desc.str.lower()
prods_ingr['new_desc'] = prods_ingr['desc'].map(lambda x: re.sub(r'\W+', ' ', x))
prods_ingr['wt_desc'] = prods_ingr['new_desc'].map(lambda x: re.sub("\d+(\s|g|mth|cm|hr|gram|s|kg|kilograms|cl|ml|ltr|pk| pk|gms|s|x|pack|l|lb|mg|d|in|yo|p|c|mm|qp|er|m|vol|h|oz|th|w|yr|tbsp)", '', x))
prods_ingr['nr_desc'] = prods_ingr['wt_desc'].map(lambda x: re.sub('^\d+\s|\s\d+\s|\s\d+$', ' ', x))
# prods_ingr['final_desc'] = prods_ingr.nr_desc.str.split(' ', 1,expand = True)[1]
prods_ingr
# prods['final_desc'] = prods['wt_desc'].map(lambda x: re.sub('^\d+\s|\s\d+\s|\s\d+$', '', x))

# prods.drop_duplicates(subset = 'prod',keep = 'last')
# prods
# # prods.loc[prods['final_desc'] == '         ']
# # prods.final_desc.describe()

# prods_1 = prods[~prods.final_desc.isin([None])]
# prods_1


,tpn,prod_group_desc,category,very_long_text,short_desc,desc,new_desc,wt_desc,nr_desc,final_desc
0,56170000,HARTLEYS NOADDEDSUGAR RDY TO EATJLY ORNG 115G,GROCER Y.,Flavouring Gelling Agents: Locust Bean Gum; Xa...,flavouring gelling agents: locust bean gum; xa...,flavouring gelling agents : locust bean gum ; ...,flavouring gelling agents locust bean gum xant...,flavouring gelling agents locust bean gum xant...,flavouring gelling agents locust bean gum xant...,gelling agents locust bean gum xanthan gum gel...
1,56930000,TESCO 10 VEGETABLE STOCK CUBES 100G,GROCER Y.,,,,,,,None
2,70780000,TISSERAND ESS SURVIVAL KIT,HB&WELLNES S.,,,,,,,None
3,77110000,JPG LE MALE 75ML EDT GIFTSET,HB&WELLNES S.,,,,,,,None
4,80330000,PORK SCRATCHINGSHAND COOKED 50G (P),IMPULS E.,Yeast Extract Powder Herb (Thyme) Salt Pork Ri...,yeast extract powder herb (thyme) salt pork ri...,yeast extract powder herb ( thyme ) salt pork ...,yeast extract powder herb thyme salt pork rind...,yeast extract powder herb thyme salt pork rind...,yeast extract powder herb thyme salt pork rind...,extract powder herb thyme salt pork rind rice ...
5,80860000,PARACETAMOL ORAL SUSPENSION 200ML (P),HB&WELLNES S.,,,,,,,None
6,50042300,SCHWARTZ ITALIANHERB SEASONING 11G JAR,GROCER Y.,Sage Parsley Thyme Bay Leaves Basil (5%) Black...,sage parsley thyme bay leaves basil (5%) black...,sage parsley thyme bay leaves basil ( 5 % ) bl...,sage parsley thyme bay leaves basil 5 black pe...,sage parsley thyme bay leaves basil black pepp...,sage parsley thyme bay leaves basil black pepp...,parsley thyme bay leaves basil black pepper or...
7,50043800,KNORR CHICKEN STOCK CUBES 8 X 10G,GROCER Y.,Antioxidant (Extracts of Rosemary) Lemon Juice...,antioxidant (extracts of rosemary) lemon juice...,antioxidant ( extracts of rosemary ) lemon jui...,antioxidant extracts of rosemary lemon juice p...,antioxidant extracts of rosemary lemon juice p...,antioxidant extracts of rosemary lemon juice p...,extracts of rosemary lemon juice powder vegeta...
8,50071000,TESCO READY SALTED CRISPS 6X25G,IMPULS E.,,,,,,,None
9,50324900,TESCO L/CAL INDIAN TONIC WATER 1LITR,IMPULS E.,,,,,,,None


In [8]:

# from nltk.corpus import stopwords
# print(stopwords.words("english"))



from sklearn.feature_extraction.text import TfidfVectorizer
vector_new = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = "english")
# ,\
#                              max_features = 40000)
# ,\
#                              ngram_range=(1,2))

product_bag = vector_new.fit_transform(prods_ingr.nr_desc).toarray()
product_bag.shape


vector_new.get_feature_names()

# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(analyzer = "word",   \
#                              tokenizer = None,    \
#                              preprocessor = None, \
#                              stop_words = "english",   \
#                              max_features = 40000,\
#                              ngram_range =(1,2)) 

# product_bag_words = vectorizer.fit_transform(prods_1.final_desc).toarray()
# product_bag_words.shape


# vectorizer.get_feature_names()


['1149666a',
 '141ii',
 '150a',
 '152a',
 '15eo',
 '160a',
 '160b',
 '160e',
 '1equivalent',
 '1from',
 '1rainforest',
 '2bromo',
 '2kci',
 '2kcl',
 '2nitro',
 '2nitropropane',
 '331iii',
 '341iii',
 '3a',
 '40b',
 '450i',
 '450vii',
 '452i',
 '470b',
 '472e',
 '4r',
 '500i',
 '500ii',
 '503ii',
 '503iii',
 '553b',
 '5z',
 '7a',
 'aa',
 'aakg',
 'aapergillus',
 'abalone',
 'abeille',
 'aberdeen',
 'abies',
 'abs',
 'absolutely',
 'absorbed',
 'absorbent',
 'absorbic',
 'absorption',
 'abv',
 'abyssinian',
 'abyssinica',
 'ac',
 'acacia',
 'acaciasenegal',
 'acaciasenegalgum',
 'acai',
 'accelerator',
 'accinium',
 'accordance',
 'according',
 'ace',
 'acer',
 'acerola',
 'acerylates',
 'aceslfame',
 'acesulfam',
 'acesulfame',
 'acesulfarne',
 'acesulphame',
 'aceta',
 'acetal',
 'acetamide',
 'acetaminophen',
 'acetate',
 'acetates',
 'acetic',
 'aceto',
 'acetone',
 'acetonide',
 'acetophenone',
 'acetoxy',
 'acetyl',
 'acetylacetonate',
 'acetylarginyltryptophyl',
 'acetylaringyltry

In [11]:
# import pandas as pd

# products_ing= pd.concat([prods_ingr.tpn,prods_ingr.prod_group_desc, prods_ingr.nr_desc,prods_ingr.category,
#                                pd.DataFrame(product_bag, 
#                                             columns= vector_new.get_feature_names(), index = prods_ingr.index)], axis=1)

# prod_shape_ingr= pd.DataFrame(product_bag,columns= vector_new.get_feature_names(), index = prods_ingr.index)

products_ing.head()

# products.loc[products['30'] == 1]
# products_1 = products

# products_1.drop('tpn', axis=1)
# products_1.drop('short_desc',axis = 1)
# prod_shape_ingr.sum().sort_values(ascending=False).head(100)

,tpn,prod_group_desc,nr_desc,category,1149666a,141ii,150a,152a,15eo,160a,...,zinziber,zircon,zirconium,zizannoides,zizanoides,zoelites,zucchini,zygis,zymatech,zywiecka
0,56170000,HARTLEYS NOADDEDSUGAR RDY TO EATJLY ORNG 115G,flavouring gelling agents locust bean gum xant...,GROCER Y.,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,56930000,TESCO 10 VEGETABLE STOCK CUBES 100G,,GROCER Y.,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,70780000,TISSERAND ESS SURVIVAL KIT,,HB&WELLNES S.,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,77110000,JPG LE MALE 75ML EDT GIFTSET,,HB&WELLNES S.,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,80330000,PORK SCRATCHINGSHAND COOKED 50G (P),yeast extract powder herb thyme salt pork rind...,IMPULS E.,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# import time as time 
# import scipy.sparse
# import random
# import itertools
# import numpy as np

# # df = prods_1.sample(frac=1).reset_index(drop=True)
# # df

# tfidf = vector_new.fit_transform(prods_1.final_desc).toarray()
# # # tfidf

# # from sklearn.metrics.pairwise import linear_kernel
# # from sklearn.metrics.pairwise import cosine_similarity
# # from scipy import sparse 
# # # TIP : to record runtime stats 

# # A=sparse.csr_matrix(tfidf)

start_time = time.time()

b =[]
# top_10 =[]
# for i, x in enumerate(tfidf):
for i in range(30000): 
#     v = tfidf[i].reshape(1,-1)
    a = linear_kernel(tfidf[i], tfidf).flatten()
#     a = scipy.spatial.distance.cdist(v, tfidf, 'cosine')
#     a = cosine_similarity(tfidf[i], tfidf).flatten()
    b.append(a)
#     r = a.argsort()[:-11:-1]
#     top_10.append(r)
#     # base similarity matrix (all dot products)
# # replace this with A.dot(A.T).todense() for sparse representation
#     similarity = A.dot(A[i,:].T).todense()
# # squared magnitude of preference vectors (number of occurrences)
#     square_mag = np.diag(similarity)
# # inverse squared magnitude
#     inv_square_mag = 1 / square_mag
# # if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
#     inv_square_mag[np.isinf(inv_square_mag)] = 0
# # inverse of the magnitude
#     inv_mag = np.sqrt(inv_square_mag)
# # cosine similarity (elementwise multiply by inverse magnitudes)
#     cosine = similarity * inv_mag
# #     cosine = cosine.T * inv_mag 
#     b.append(cosine)

print("--- RUNTIME : %s seconds ---" % (time.time() - start_time))

# b

c = np.array(b)
# # # c.shape
# # # c
# # # prods_1
# start_time = time.time()

a =[]
for (x,y), k in np.ndenumerate(c):
    if x < 30001 and y < 60144 and k > 0.499 and x!=y and (prods_1.iloc[x][7] != prods_1.iloc[y][7]):
        s = (prods_1.iloc[x][9], prods_1.iloc[x][5], prods_1.iloc[y][9], prods_1.iloc[y][5],prods_1.iloc[x][14],prods_1.iloc[y][14],k)
        a.append(s)     

m = pd.DataFrame(a)
# print("--- RUNTIME : %s seconds ---" % (time.time() - start_time))

# m
m.to_csv('/nfs/science/shared/ipythonNotebooks/anantk/prod_match_up4.csv', sep=',')
# /nfs/science/shared/ipythonNotebooks/anantk


/usr/lib64/python3.4/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lib64/python3.4/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lib64/python3.4/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lib64/python3.4/s

In [22]:
# bb = pd.read_sas('/nfs/science/shared/ipythonNotebooks/anantk/brandbank_product_claims_v3.sas7bdat', )
print(bb.head())


NameError: name 'bb' is not defined